# gRPC

## 参考文献

* https://github.com/grpc/grpc
* https://grpc.io/docs/languages/python/quickstart/

## gRPC Python Demo

### 安装依赖

```shell
pip install grpcio
pip install grpcio-tools
```

### 定义proto

```proto
syntax = "proto3";

option java_multiple_files = true;
option java_package = "io.grpc.examples.helloworld";
option java_outer_classname = "HelloWorldProto";
option objc_class_prefix = "HLW";

package helloworld;

// The greeting service definition.
service Greeter {
  // Sends a greeting
  rpc SayHello (HelloRequest) returns (HelloReply) {}
}

// The request message containing the user's name.
message HelloRequest {
  string name = 1;
}

// The response message containing the greetings
message HelloReply {
  string message = 1;
}
```

### 生成代码

```shell
python -m grpc_tools.protoc -I . --python_out=. --grpc_python_out=. helloworld.proto
```

### grpc server

```python
from concurrent import futures
import time

import grpc

import helloworld_pb2
import helloworld_pb2_grpc

_ONE_DAY_IN_SECONDS = 60 * 60 * 24


class Greeter(helloworld_pb2_grpc.GreeterServicer):

    def SayHello(self, request, context):
        return helloworld_pb2.HelloReply(message='Hello, %s!' % request.name)


def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    helloworld_pb2_grpc.add_GreeterServicer_to_server(Greeter(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    try:
        while True:
            time.sleep(_ONE_DAY_IN_SECONDS)
    except KeyboardInterrupt:
        server.stop(0)


if __name__ == '__main__':
    serve()

```

### grpc client

```python
from __future__ import print_function

import grpc

import helloworld_pb2
import helloworld_pb2_grpc


def run():
    # NOTE(gRPC Python Team): .close() is possible on a channel and should be
    # used in circumstances in which the with statement does not fit the needs
    # of the code.
    with grpc.insecure_channel('localhost:50051') as channel:
        counter = 9
        stub = helloworld_pb2_grpc.GreeterStub(channel)
        while counter >= 0:
            response = stub.SayHello(helloworld_pb2.HelloRequest(name='you'))
            print(f"counter: {counter}, Greeter client received: {response}")
            counter -= 1


if __name__ == '__main__':
    run()
```